In [11]:
# EDA libraries
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport 

# Mounting the data dir 
from pathlib import Path
data_dir_interim = Path('.').resolve().parent / "data" / "interim"

In [12]:
# Load interim data 
df = pd.read_csv(data_dir_interim / "cleaned_health_insurance_us_v1.csv")
print(f'Shape of the data: {df.shape}')
print(f'Preview of the data:\n {df.head()}')

Shape of the data: (1337, 7)
Preview of the data:
    age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520
